In [3]:
from IPython.display import display, HTML

display(HTML(data="""

<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

%reload_ext jupyternotify

#%notify

<IPython.core.display.Javascript object>

In [15]:
import numpy as np
import sys
from qutip import *
import matplotlib.pyplot as plt
from pathlib import Path

from scipy import linalg as la

parent_path = ".."  #str(Path(__file__).resolve().parents[1])
sys.path.append(parent_path)

import pysqkit


import time
import os

from pysqkit.solvers.solvkit import integrate

%matplotlib notebook
plt.rcParams['figure.figsize'] = (10, 7)

SyntaxError: invalid syntax (system.py, line 63)

In [ ]:
from math import pi

h = 6.62607 * 10**(-34)
h_bar = h/(2*pi)
e = 1.6071766*10**(-19)
phi_0 = h/(2*e)


In [ ]:
N = 100

# Init

In [ ]:
J_c = 300*10**6 *h
lam = .5

In [ ]:
t = np.linspace(0, 400, 2000) * 10**(-9) #in nano seconds

$$\hat{H} = \hat{H}^{(0)}_A + \hat{H}^{(0)}_B + \hat{V} + \hat{H}_{drive}$$

## flxA



$$\hat{H}^{(0)}_A = 4 E_{C,A} \cdot \hat{n}^2_A + \frac{1}{2}E_{L,A}\cdot\hat{\varphi}^2_A - E_{J,A}\cos\left(\hat{\varphi}_A - \phi_{ext,A}\right)$$

In [ ]:
e_c_A = 1*10**9 *h
e_l_A = 1.5*10**9 *h
e_j_A = 3.8*10**9 *h
  
flux_A = 1/2  # 1/2 -> pi dans le cos

flxA = pysqkit.qubits.Fluxonium(label = "QBA",
                               charge_energy=e_c_A,
                               induct_energy=e_l_A,
                               joseph_energy=e_j_A,
                               flux=flux_A,
                               basis=None,
                               dim_hilbert=N)

In [ ]:
eig_enegies, eig_states = flxA.eig_states(levels=5)
flxA.basis.transform(eig_states)

In [ ]:
#Bare energies

bare_E_A = flxA.eig_energies()
bare_E_A/(h*10**9)

In [ ]:
# transitions freqs

omega_01_A = bare_E_A[1] - bare_E_A[0]
omega_12_A = bare_E_A[2] - bare_E_A[1]

omega_01_A/(h*10**9) , omega_12_A/(h*10**9) 

In [ ]:
# mat elements :

n_10_A = np.abs(flxA.mat_elements('charge_op', levels = 5)).data[1,0]
n_01_A = np.abs(flxA.mat_elements('charge_op', levels = 5)).data[0,1]

np.round(np.abs(flxA.mat_elements('charge_op', levels = 5)).data, 2), n_01_A, n_10_A

### Rq)

In [ ]:
plt.figure()
for i in range(len(bare_E_A)):
    plt.scatter(i, bare_E_A[i])

In [ ]:
flxA.charge_op(as_qobj = True)

In [ ]:
flxA.charge_zpf*(create(100) - destroy(100))*1j

In [ ]:
flxA.charge_op(as_qobj = True)

In [ ]:
flxA.hamiltonian(as_qobj = True)/(h*10**9)

## flx2

$$\hat{H}^{(0)}_B = 4 E_{C,B} \cdot \hat{n}^2_B + \frac{1}{2}E_{L,B}\cdot\hat{\varphi}^2_B - E_{J,B}\cos\left(\hat{\varphi}_B - \phi_{ext,B}\right)$$

In [ ]:
e_c_B = 1*10**9 *h
e_l_B = .9*10**9 *h
e_j_B = 3*10**9 *h

flux_B = 1/2

flxB = pysqkit.qubits.Fluxonium(label = "QBB",
                               charge_energy=e_c_B,
                               induct_energy=e_l_B,
                               joseph_energy=e_j_B,
                               flux=flux_B,
                               basis=None,
                               dim_hilbert=N)

In [ ]:
eig_enegies, eig_states = flxB.eig_states(levels=5)
flxB.basis.transform(eig_states)

In [ ]:
#Bare energies

bare_E_B = flxB.eig_energies()
bare_E_B/(h*10**9)

In [ ]:
# transitions freqs

omega_01_B = bare_E_B[1] - bare_E_B[0]
omega_12_B = bare_E_B[2] - bare_E_B[1]

omega_01_B/(h*10**9) , omega_12_B/(h*10**9) 

In [ ]:
# mat elements :

n_10_B = np.abs(flxB.mat_elements('charge_op', levels = 5)).data[1,0]
n_01_B = np.abs(flxB.mat_elements('charge_op', levels = 5)).data[0,1]

np.round(np.abs(flxB.mat_elements('charge_op', levels = 5)).data, 2), n_01_B, n_10_B

## coupling

$$\hat{V} = J_C \cdot \hat{n}_A \cdot \hat{n}_B$$

In [ ]:
g = J_c#J_c/(4*e**2)

In [ ]:
duo = flxA.couple_to(flxB, pysqkit.couplers.capacitive_coupling, strength = g)

In [ ]:
duo.eig_energies()/(h*10**9)

In [ ]:
duo.hamiltonian()

### rqs

In [ ]:
duo.bare_hamiltonian(), duo.int_hamiltonian()

In [ ]:
#Bare energies

bare_E_duo = la.eig(duo.bare_hamiltonian())[0].real/h #in Hz
bare_E_duo.sort()
bare_E_duo = bare_E_duo[:5]*10**(-9)  #in GHz

int_E_duo = la.eig(duo.int_hamiltonian())[0].real/h
int_E_duo.sort()
int_E_duo = int_E_duo[:5]*10**(-9)

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (10,5))

axs[0].scatter(range(len(bare_E_duo)), bare_E_duo, c = 'r', label = 'bare')    
axs[1].scatter(range(len(int_E_duo)), int_E_duo, c = 'b', label = 'int')

axs[0].legend()
axs[1].legend()

axs[0].set_ylabel("E (GHz)")
axs[1].set_ylabel("E (GHz)")

In [ ]:
E_duo = bare_E_duo + int_E_duo
E_duo.sort()

plt.figure()
plt.scatter(range(len(E_duo)), E_duo)

plt.annotate("  |00>", (0,E_duo[0]))
plt.annotate("  |01>", (1,E_duo[1]))
plt.annotate("  |10>", (2,E_duo[2]))
plt.annotate("  |11>", (3,E_duo[3]))
plt.annotate("  |??>", (4,E_duo[4]))




plt.ylabel('E (GHz)')

In [ ]:
print("Ground =", E_duo[0], "GHz\n1st state =", E_duo[1], "GHz\n2nd state =", E_duo[2], "GHz" )

print("\n\nomega 00->01 =", E_duo[1]-E_duo[0], "GHz\nomega 01->10 =", E_duo[2] - E_duo[1], "GHz\nomega 10->11 =", E_duo[3]-E_duo[2], "GHz"
     )

In [ ]:
duo.hamiltonian()

## Drive

$$\hat{H}_{drive} = 2  f\left( t \right)  \cos \left( \omega_d t + \gamma_d\right) \cdot \left( \eta_A \cdot \hat{n}_A + \eta_B \cdot \hat{n}_B \right)$$

$$f\left( t \right) = f_0 \cdot \dfrac{f\left( t \right)}{f_0}$$

In [ ]:
f_0 = lam/2 * np.abs(omega_01_A - omega_01_B) / (n_10_A)

In [ ]:
t_rise = 25*10**(-9)
sigma = t_rise/2

f_by_f0 = np.exp(- (t-t_rise)**2 / (2*sigma**2)) - np.exp(-t_rise**2 /(2*sigma**2))

In [ ]:
f = f_0 * f_by_f0

omega_d = 2*pi * 1 *10**9
gamma_d = 0

eta_1 = 1
eta_2 = 1

In [ ]:
n_A = flxA.charge_op(as_qobj = True)
n_B = flxB.charge_op(as_qobj = True)

In [ ]:
H_drive = []

for i in range(len(t)):
    H_drive.append(2 * f[i] * np.cos(omega_d*t[i]) * (n_1 + n_2))

In [ ]:
drive = n_A + n_B

In [ ]:
pulse = 2 * f * np.cos(omega_d*t)

### Rq)

In [ ]:
n_1+n_2 #Rq) only different parity term

In [ ]:
n_1*n_2 #Rq) only same parity terms

# Simu

In [ ]:
#duo hamil to qobj
dim = N

duo_hamil = Qobj(
                inpt=duo.hamiltonian(),
                dims=[dim, dim],
                shape=duo.hamiltonian(),
                type='oper',
                isherm=True)

In [ ]:
time =  t
state_init = fock(N,0)
hamil = duo_hamil
drive = [drive*np.ones(len(t))]
pulse = [pulse]
jump = []
solver = pysqkit.solvers.solvkit.supported_solvers[0]

In [ ]:
res = integrate(time, state_init, hamil, drive, pulse, jump, solver)